# Plan.

 Drop rows if needed.
 Shrink the dataset to 15-20k rows according to popularity.
 Drop columns if needed, select only those that will be interesting for clusterin model.


Quiz will ask a person to pick few favourite movies from a list of 20-50 most popular ones we have.

1) Network based
 First plan is to create a network of viewers and cluster according to movie taste. That will be done with 'ratings' dataset. Then once I have new customer movie taste I will try to find a cluster of similar customers and recommend movies he liked.

Example: there will be cluster of customers that gave high scores to comedies and if new person happen to like comedies then he will belong to this cluster and movies people in this cluster liked will be recommended.
 
2) Similar movies?
 Second one, I have movies he liked I can try to find similar movies to them and as an output get say 10 similar movies (then recommended movie picked at random say). 

Example:?

In [23]:
import pandas as pd
import numpy as np
import json
import seaborn as sns

from sklearn.mixture import DPGMM

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df = pd.read_csv("movies_metadata.csv")
ratingsdf = pd.read_csv("ratings_small.csv") #will work on small one 
keywordsdf = pd.read_csv("keywords.csv")     #for computational reasons  
creditsdf = pd.read_csv("credits.csv")

C:\Users\Mikolaj\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

Defining few functions to be used later on.

In [2]:
def description(dataframe):
    for col in dataframe:
        print(dataframe[col].describe())

Here I am making subsets as I will be working only on 'df', Greg will be doing NLP on "df_nlp". Also I will merge later 'df_json' once I find out how to encode it.

In [24]:
df = df.replace({"overview": {'No overview found.': None}})
df = df.replace({"overview": {'No Overview': None}})

In [11]:
df_nlp = df[["id", "overview","tagline", "title", "original_title"]]
df_json = df[["id", "belongs_to_collection", "genres", "production_countries", "spoken_languages", "production_companies"]]

drop_cols = ["belongs_to_collection", "genres", "production_countries",
              "spoken_languages", 'production_companies', "overview","tagline", "title", "original_title"]

df = df.drop(drop_cols, axis=1)

In [12]:
df.columns
df.head(5)
df.info()

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'popularity', 'poster_path', 'release_date', 'revenue', 'runtime',
       'status', 'video', 'vote_average', 'vote_count'],
      dtype='object')

,adult,budget,homepage,id,imdb_id,original_language,popularity,poster_path,release_date,revenue,runtime,status,video,vote_average,vote_count
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,False,7.7,5415.0
1,False,65000000,NaN,8844,tt0113497,en,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,False,6.9,2413.0
2,False,0,NaN,15602,tt0113228,en,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,False,6.5,92.0
3,False,16000000,NaN,31357,tt0114885,en,3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,False,6.1,34.0
4,False,0,NaN,11862,tt0113041,en,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,False,5.7,173.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 15 columns):
adult                45466 non-null object
budget               45466 non-null object
homepage             7782 non-null object
id                   45466 non-null object
imdb_id              45449 non-null object
original_language    45455 non-null object
popularity           45461 non-null object
poster_path          45080 non-null object
release_date         45379 non-null object
revenue              45460 non-null float64
runtime              45203 non-null float64
status               45379 non-null object
video                45460 non-null object
vote_average         45460 non-null float64
vote_count           45460 non-null float64
dtypes: float64(4), object(11)
memory usage: 5.2+ MB


I will drop few columns to narrow it down to smaller subset.

In [13]:
dropped = df[['homepage', 'imdb_id', 'video', 'poster_path', 'status']]
df = df.drop(['homepage', 'imdb_id', 'video', 'poster_path', 'status'], axis=1)

# Cleaning metadata. 

In [14]:
df.head(5)

,adult,budget,id,original_language,popularity,release_date,revenue,runtime,vote_average,vote_count
0,False,30000000,862,en,21.9469,1995-10-30,373554033.0,81.0,7.7,5415.0
1,False,65000000,8844,en,17.0155,1995-12-15,262797249.0,104.0,6.9,2413.0
2,False,0,15602,en,11.7129,1995-12-22,0.0,101.0,6.5,92.0
3,False,16000000,31357,en,3.85949,1995-12-22,81452156.0,127.0,6.1,34.0
4,False,0,11862,en,8.38752,1995-02-10,76578911.0,106.0,5.7,173.0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 10 columns):
adult                45466 non-null object
budget               45466 non-null object
id                   45466 non-null object
original_language    45455 non-null object
popularity           45461 non-null object
release_date         45379 non-null object
revenue              45460 non-null float64
runtime              45203 non-null float64
vote_average         45460 non-null float64
vote_count           45460 non-null float64
dtypes: float64(4), object(6)
memory usage: 3.5+ MB


In [17]:
df["popularity"].unique()

array([21.946943, 17.015539, 11.7129, ..., '0.903007', '0.003503',
       '0.163015'], dtype=object)

In [48]:
def fn(df):
    listek = []
    for value in df["popularity"]:
        if type(value)==str:
            listek.append(value)
        listek = [float(x) for x in listek]
    return listek

fn(df)

ValueError: could not convert string to float: 'Beware Of Frost Bites'